In [18]:
## Primero se importan las librerías necesarias

import subprocess              # para instalar paquetes
import sys                     # para manejar el entorno de Python
import pandas as pd            # manipulación de datos
import spacy                   # procesamiento de lenguaje natural
from tqdm.auto import tqdm     # barras de progreso

# Instalar spaCy si no está instalado
subprocess.check_call([sys.executable, "-m", "pip", "install", "spacy", "-q"])

# Descargar el modelo de spaCy para español
subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_lg", "-q"])

print("Librerías importadas correctamente.")

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
Librerías importadas correctamente.


In [19]:
# Cargar el dataset

df = pd.read_csv("/workspaces/TratamientoAutomaticoDelLenguaje/Datos/dataset_proyecto_chile_septiembre2025.csv")

print(df.head())
print("Dataset cargado correctamente.")

        _id                    _index _score country country.keyword  \
0  51150648  news_fondecyt_1250207_fr      -   chile           chile   
1  51150651  news_fondecyt_1250207_fr      -   chile           chile   
2  51150653  news_fondecyt_1250207_fr      -   chile           chile   
3  51150655  news_fondecyt_1250207_fr      -   chile           chile   
4  51150657  news_fondecyt_1250207_fr      -   chile           chile   

                          date     id_news media_outlet media_outlet.keyword  \
0  Sep 24, 2025 @ 00:00:00.000  51,150,648         emol                 emol   
1  Sep 24, 2025 @ 00:00:00.000  51,150,651         emol                 emol   
2  Sep 23, 2025 @ 00:00:00.000  51,150,653         emol                 emol   
3  Sep 25, 2025 @ 00:00:00.000  51,150,655         emol                 emol   
4  Sep 25, 2025 @ 00:00:00.000  51,150,657         emol                 emol   

                                                text  \
0  En un incendio terminó el "

In [20]:
# Limpieza de Informacion

def limpiar_dataframe_basico(df_entrada):
    """Selecciona columnas relevantes, filtra textos cortos/nulos y normaliza la fecha."""
    # Seleccionar columnas y copiar el DataFrame
    df_limpio = df_entrada[['date', 'media_outlet', 'title', 'text', 'url']].copy() 

    # Eliminar filas sin título o texto
    df_limpio = df_limpio.dropna(subset=['text', 'title'])

    # Quitar textos muy cortos o basura
    df_limpio = df_limpio[df_limpio['text'].str.len() > 200].copy()

    # parsear y ordenar la fecha ("Sep 24, 2025 @ 00:00:00.000")
    def parsear_fecha(cadena_fecha):
        if pd.isna(cadena_fecha):
            return pd.NaT
        try:
            parte_fecha = str(cadena_fecha).split('@')[0].strip()
            return pd.to_datetime(parte_fecha, format='%b %d, %Y', errors='coerce')
        except Exception:
            return pd.NaT

    # Aplicar parseo y eliminar filas sin fecha válida
    df_limpio['date'] = df_limpio['date'].apply(parsear_fecha)
    df_limpio = df_limpio.dropna(subset=['date'])

    # Columna con fecha en formato YYYY-MM-DD
    df_limpio['fecha_yyyy_mm_dd'] = df_limpio['date'].dt.strftime('%Y-%m-%d')

    print(f"Filas después de limpieza: {len(df_limpio)}")
    return df_limpio

df_limpio = limpiar_dataframe_basico(df)
print(df_limpio.head())

Filas después de limpieza: 10287
        date media_outlet                                              title  \
0 2025-09-24         emol  "Banderazo" de hinchas de la U termina en desm...   
1 2025-09-24         emol  Tras nuevo femicidio: Piden poner urgencia a p...   
2 2025-09-23         emol  Detienen a dos alumnos tras ser sorprendidos c...   
3 2025-09-25         emol  Metro normaliza servicio en Línea 1 tras cierr...   
4 2025-09-25         emol  Seguridad en Mundial Sub 20: Roban a funcionar...   

                                                text  \
0  En un incendio terminó el "banderazo" convocad...   
1  Tras registrarse un nuevo caso de femicidio en...   
2  Carabineros detuvo esta mañana a  dos adolesce...   
3  La mañana de este jueves el Metro de Santiago ...   
4  Este sábado comienza el  Mundial Sub 20 en Chi...   

                                                 url fecha_yyyy_mm_dd  
0  https://www.emol.com/noticias/Nacional/2025/09...       2025-09-24  
1  ht

In [ ]:
# Se deefine los elementos para la extracción de NER en batch
nlp = spacy.load("es_core_news_lg", exclude=[
    "tagger", 
    "parser", 
    "attribute_ruler",
    "lemmatizer"
])

# Función para procesar textos en batch
def procesar_batch(textos):
    personas_list = []
    lugares_list = []
    organizaciones_list = []
    
    for doc in tqdm(nlp.pipe(textos, batch_size=256, n_process=1),
                    total=len(textos),
                    desc="Procesando NER"):
        
        personas = {ent.text.strip() for ent in doc.ents if ent.label_ == "PER"}
        lugares = {ent.text.strip() for ent in doc.ents if ent.label_ in ["LOC", "GPE"]}
        organizaciones = {ent.text.strip() for ent in doc.ents if ent.label_ == "ORG"}

        personas_list.append(list(personas))
        lugares_list.append(list(lugares))
        organizaciones_list.append(list(organizaciones))

    return personas_list, lugares_list, organizaciones_list


# Extracción de entidades NER en batch
print("Extrayendo entidades NER en batch…", flush=True)

textos = df_limpio["text"].tolist()
personas, lugares, organizaciones = procesar_batch(textos)

# Agregar las nuevas columnas al DataFrame limpio
df_limpio["personas"] = personas
df_limpio["lugares"] = lugares
df_limpio["organizaciones"] = organizaciones

Extrayendo entidades NER en batch…


Procesando NER: 100%|██████████| 10287/10287 [08:29<00:00, 20.20it/s]


In [22]:
# Detección de regiones de Chile
regiones_cl = [
    "Arica y Parinacota", "Tarapacá", "Antofagasta", "Atacama", "Coquimbo",
    "Valparaíso", "Metropolitana", "O'Higgins", "Maule", "Ñuble", "Biobío",
    "Araucanía", "Los Ríos", "Los Lagos", "Aaysén", "Magallanes",
    "Iquique", "Antofagasta", "Santiago", "Concepción", "Temuco", "Puerto Montt"
]

# Función para detectarlas
def detectar_regiones(row):
    texto = (str(row['title']) + " " + str(row['text']) + " " + " ".join(row['lugares'])).lower()
    encontradas = [r for r in regiones_cl if r.lower() in texto]
    return list(set(encontradas))

df_limpio['regiones'] = df_limpio.apply(detectar_regiones, axis=1)

print("Regiones detectadas correctamente.")

# Contabilidad de filas sin regiones detectadas
sin_region = df_limpio[df_limpio['regiones'].apply(lambda x: len(x) == 0)]
print(f"Filas sin regiones detectadas: {len(sin_region)}")

print(df_limpio[['title', 'regiones']].head(10))

Regiones detectadas correctamente.
Filas sin regiones detectadas: 5943
                                               title  \
0  "Banderazo" de hinchas de la U termina en desm...   
1  Tras nuevo femicidio: Piden poner urgencia a p...   
2  Detienen a dos alumnos tras ser sorprendidos c...   
3  Metro normaliza servicio en Línea 1 tras cierr...   
4  Seguridad en Mundial Sub 20: Roban a funcionar...   
5  Encerrona en La Reina: Ocho sujetos roban vehí...   
6  Dureza con Trump y Milei y loas a Giorgio Jack...   
7  Conservador de Bienes Raíces inscribe compra d...   
8  Violento episodio: Hombre mata a su pareja de ...   
9  Homicidio en Estación Central: Desconocido acr...   

                    regiones  
0                 [Coquimbo]  
1                 [Santiago]  
2                   [Biobío]  
3                 [Santiago]  
4     [Metropolitana, Maule]  
5            [Metropolitana]  
6                         []  
7                 [Santiago]  
8  [Metropolitana, Santiago]  
9 

In [23]:
# guardar el DataFrame enriquecido en JSON
df_final = df_limpio[[
    'title', 'text', 'media_outlet', 'fecha_yyyy_mm_dd', 'url',
    'personas', 'lugares', 'organizaciones', 'regiones'
]].copy()

# Renombrar la columna de fecha para el JSON final
df_final = df_final.rename(columns={'fecha_yyyy_mm_dd': 'fecha'})

# Definir la ruta de salida
output_path = "Datos/noticias_enriquecidas_2025.json"
df_final.to_json(output_path, orient="records", force_ascii=False, indent=2)

print(f"\n¡TODO LISTO! {len(df_final)} noticias enriquecidas guardadas en:")
print(output_path)


¡TODO LISTO! 10287 noticias enriquecidas guardadas en:
Datos/noticias_enriquecidas_2025.json
